In [1]:
from collections import defaultdict
from fractions import Fraction

In [2]:
# Class to represent numbers (integers and rationals)
class Num:
    def __init__(self, value):
        if isinstance(value, int) or isinstance(value, Fraction):
            self.value = Fraction(value)
        else:
            raise ValueError("Only integers or rational numbers are allowed in Num class.")

    def __str__(self):
        return str(self.value)

    def __repr__(self):
        return f"Num({self.value})"

# Class to represent symbols (like variables in polynomials)
class Sym:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def __repr__(self):
        return f"Sym('{self.name}')"


In [3]:
# Class to represent polynomials
class Poly:
    def __init__(self, terms=None):
        # Terms are represented as a dictionary with symbols as keys and coefficients as values
        if terms is None:
            self.terms = defaultdict(Num)
        else:
            self.terms = defaultdict(Num, terms)

    def __str__(self):
        return ' + '.join(f"{v}*{k}" if k else str(v) for k, v in self.terms.items())

    def __repr__(self):
        return f"Poly({dict(self.terms)})"

    # Polynomial addition
    def add(self, other):
        result = Poly(self.terms.copy())  # Start with a copy of the current polynomial
        for sym, coeff in other.terms.items():
            if sym in result.terms:
                result.terms[sym] = Num(result.terms[sym].value + coeff.value)  # Add coefficients
            else:
                result.terms[sym] = coeff  # Add new term
        return result

    # Polynomial subtraction
    def sub(self, other):
        result = Poly(self.terms.copy())
        for sym, coeff in other.terms.items():
            if sym in result.terms:
                result.terms[sym] = Num(result.terms[sym].value - coeff.value)  # Subtract coefficients
            else:
                result.terms[sym] = Num(-coeff.value)  # Add negative term
        return result

    # Polynomial multiplication
    # def mul(self, other):
    #     result = Poly()
    #     for sym1, coeff1 in self.terms.items():
    #         for sym2, coeff2 in other.terms.items():
    #             new_sym = sym1 + sym2 if sym1 and sym2 else sym1 or sym2  # Concatenate symbols if both exist
    #             if new_sym in result.terms:
    #                 result.terms[new_sym] = Num(result.terms[new_sym].value + coeff1.value * coeff2.value)
    #             else:
    #                 result.terms[new_sym] = Num(coeff1.value * coeff2.value)
    #     return result
    
    def mul(self, other):
        result = Poly()
        for sym1, coeff1 in self.terms.items():
            for sym2, coeff2 in other.terms.items():
                # Concatenate symbols if both exist, otherwise use the non-empty one
                if isinstance(sym1, Sym) and isinstance(sym2, Sym):
                    new_sym = Sym(str(sym1) + '*' + str(sym2))  # Concatenate symbol names
                else:
                    new_sym = sym1 or sym2  # Use non-empty symbol if only one exists

                if new_sym in result.terms:
                    result.terms[new_sym] = Num(result.terms[new_sym].value + coeff1.value * coeff2.value)
                else:
                    result.terms[new_sym] = Num(coeff1.value * coeff2.value)
        return result
    
    def substitute(self, old_sym, new_expr):
        result = Poly()
        for sym, coeff in self.terms.items():
            # If the current term contains the symbol to substitute, replace it with the new expression
            if sym == old_sym:
                if isinstance(new_expr, Poly):
                    # Multiply the coefficient by the substituted polynomial if the new expression is a polynomial
                    for new_sym, new_coeff in new_expr.terms.items():
                        result = result.add(Poly({new_sym: Num(coeff.value * new_coeff.value)}))
                else:
                    result = result.add(Poly({new_expr: coeff}))  # Simply replace the symbol with the new expression
            else:
                result = result.add(Poly({sym: coeff}))  # Keep other terms unchanged
        return result

    def __repr__(self):
        return " + ".join([f"{coeff.value} * {sym}" for sym, coeff in self.terms.items()])



In [4]:
# Example usage of Num and Sym
x = Sym('x')
y = Sym('y')


# Polynomial: 2*x + 3*y
p1 = Poly({x: Num(2), y: Num(3)})

# Polynomial: 5*x + 4
p2 = Poly({x: Num(5), '': Num(4)})  # Empty string represents the constant term

p3 = Poly({x: Num(-3), y: Num(9), '':Num(-6)})

In [5]:


# Adding p1 and p2: (2*x + 3*y) + (5*x + 4)
p_add = p1.add(p2)
print(f"p1 + p2 = {p_add}")

# Subtracting p2 from p1: (2*x + 3*y) - (5*x + 4)
p_sub = p1.sub(p2)
print(f"p1 - p2 = {p_sub}")

# Multiplying p1 and p2: (2*x + 3*y) * (5*x + 4)
p_mul = p1.mul(p2)
print(f"p1 * p2 = {p_mul}")



p1 + p2 = 7*x + 3*y + 4
p1 - p2 = -3*x + 3*y + -4
p1 * p2 = 10*x*x + 8*x + 15*y*x + 12*y


In [6]:
x = Sym('x')
y = Sym('y')
a = Num(1)  # Assign numerical values for simplicity
b = Num(1)
c = Num(1)


In [7]:
# Define a helper function to handle powers
def power(sym, exp):
    if exp == 1:
        return sym
    else:
        return Sym(f"{sym.name}^{exp}")

In [8]:
V_xy = Poly({
    power(x, 2): a,x: b,y: c         
})

In [9]:
print(V_xy)

1*x^2 + 1*x + 1*y


In [12]:
def post_V(V, x_val):
    V1 = V.substitute(x, x_val - Num(2))  # Substitute x - 2 in V
    V2 = V.substitute(x, x_val + Num(1))  # Substitute x + 1 in V
    return (V1.mul(Poly({None: Num(0.5)}))).add(V2.mul(Poly({None: Num(0.5)})))

# Compute PostV for some_value
some_value = Sym('some_value')
postV_result = post_V(V_xy, some_value)
print(f"PostV(x) = {postV_result}")

TypeError: unsupported operand type(s) for -: 'Sym' and 'Num'

In [ ]:
# Define the update function (this will be a direct substitution later)
def update_x(x_val, w):
    return x_val + w  # Example update rule

# Define PostV(x) = 0.5 * V(x - 2, a, b, c) + 0.5 * V(x + 1, a, b, c)
def post_V(V, x_val):
    V1 = V.substitute(x, x_val - 2)  # Substitute x - 2 in V
    V2 = V.substitute(x, x_val + 1)  # Substitute x + 1 in V
    return (V1.mul(Num(0.5))).add(V2.mul(Num(0.5)))  # 0.5 * V1 + 0.5 * V2

# Compute PostV for x = some_value
some_value = Sym('some_value')
postV_result = post_V(V_xy, some_value)
print(f"PostV(x) = {postV_result}")

# Define Inv(x) = [x^2 + y^2 <= 10e6]
inv_poly = Poly({
    power(x, 2): Num(1),  # x^2
    power(y, 2): Num(1)   # y^2
}).sub(Poly({None: Num(10**6)}))  # -10e6

# Display the invariant
print(f"Invariant: {inv_poly} <= 0")

# Define Target(x) = [(x-5)^2 + (y-10)^2 <= 10e2]
target_poly = Poly({
    power(x - Num(5), 2): Num(1),  # (x - 5)^2
    power(y - Num(10), 2): Num(1)  # (y - 10)^2
}).sub(Poly({None: Num(10**2)}))  # -10e2

# Display the target region
print(f"Target: {target_poly} <= 0")
